In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# 定义点云数据的输入维度和输出维度
input_dim = 3  # 每个点的特征维度
output_dim = 3  # 输出点云的特征维度（可以与输入维度相同）

# 创建一个简单的全连接神经网络类
class PointCloudFCNet(nn.Module):
    def __init__(self):
        super(PointCloudFCNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)  # 输入层到隐藏层1
        self.fc2 = nn.Linear(64, 128)  # 隐藏层1到隐藏层2
        self.fc3 = nn.Linear(128, 256)  # 隐藏层2到隐藏层3
        self.fc4 = nn.Linear(256, 128)  # 隐藏层3到隐藏层4
        self.fc5 = nn.Linear(128, output_dim)  # 隐藏层4到输出层

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)  # 输出层不使用激活函数
        return x


In [4]:
# 创建模型实例
model = PointCloudFCNet()
model

PointCloudFCNet(
  (fc1): Linear(in_features=3, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=3, bias=True)
)

In [5]:
# 定义损失函数和优化器
criterion = nn.MSELoss()  # 使用均方误差作为损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 使用Adam优化器


In [6]:
# 示例输入点云数据（每个点的特征维度为3）
input_point_cloud = torch.randn(10, input_dim)  # 10个点的点云数据
input_point_cloud

tensor([[ 0.6435,  0.3537, -0.0179],
        [-0.0982,  1.1130, -0.7449],
        [ 0.9892,  0.1201, -1.0190],
        [ 0.0074, -0.0769,  0.4797],
        [ 0.0890,  0.3836, -0.8749],
        [ 0.1149, -2.1214,  0.2283],
        [ 0.4214, -1.7091,  0.6073],
        [-0.7279,  0.7734,  1.4882],
        [-0.3199,  0.4695,  0.8873],
        [-1.5824,  0.7212,  0.1230]])

In [7]:
# 目标输出点云数据（与输入点云相同的形状）
target_point_cloud = torch.randn(10, output_dim)
target_point_cloud

tensor([[ 1.0068, -0.2279, -0.5689],
        [-0.0732,  0.0417, -0.1805],
        [ 0.4927, -1.0731, -0.6703],
        [-0.0979,  0.8071,  0.9436],
        [-0.9364, -2.0466, -0.5105],
        [ 0.6254,  0.3325, -0.2109],
        [-1.1563,  1.3631, -0.1989],
        [ 0.3121,  0.2697,  0.0866],
        [-0.0191,  1.3951,  1.1398],
        [-0.2412, -0.9598, -0.4327]])

In [8]:
# 训练模型
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output_point_cloud = model(input_point_cloud)  # 前向传播
    loss = criterion(output_point_cloud, target_point_cloud)  # 计算损失
    loss.backward()  # 反向传播
    optimizer.step()  # 优化模型参数

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [100/1000], Loss: 0.0000
Epoch [200/1000], Loss: 0.0002
Epoch [300/1000], Loss: 0.0000
Epoch [400/1000], Loss: 0.0000
Epoch [500/1000], Loss: 0.0002
Epoch [600/1000], Loss: 0.0000
Epoch [700/1000], Loss: 0.0000
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.0000
Epoch [1000/1000], Loss: 0.0000


In [10]:
# 使用训练好的模型生成新的点云数据
new_point_cloud = model(input_point_cloud)
new_point_cloud
# 输出的new_point_cloud包含了经过神经网络处理后的新点云数据

tensor([[ 1.0068, -0.2279, -0.5689],
        [-0.0732,  0.0417, -0.1805],
        [ 0.4927, -1.0731, -0.6703],
        [-0.0979,  0.8071,  0.9436],
        [-0.9364, -2.0466, -0.5105],
        [ 0.6254,  0.3325, -0.2109],
        [-1.1563,  1.3631, -0.1989],
        [ 0.3121,  0.2697,  0.0866],
        [-0.0191,  1.3951,  1.1398],
        [-0.2412, -0.9598, -0.4327]], grad_fn=<AddmmBackward0>)